In [5]:
import random
import pypokedex
import time
from bs4 import BeautifulSoup
import requests
import re

games = [
    "red-blue","yellow","gold-silver","crystal","ruby-sapphire","firered-leafgreen","emerald","diamond-pearl",
    "platinum","heartgold-soulsilver","black-white","x-y","sun-moon","sword-shield","scarlet-violet"
]

exceptions = [
    "High-Jump-Kick", "Wake-Up-Slap", "Power-Up-Punch", "Baby-Doll-Eyes", "All-Out-Pummeling", "Never-Ending-Nightmare", 
    "Black-Hole-Eclipse", "Soul-Stealing-7-Star-Strike", "10,000,000-Volt-Thunderbolt","Light-That-Burns-the-Sky", 
    "Searing-Sunraze-Smash", "Menacing-Moonraze-Maelstrom", "Let's-Snuggle-Forever", "Will-O-Wisp"
]

In [7]:
moveset = []
f_moveset = []
ff_moveset = []

occurence_count = 0
position = -1

p = input().lower()
poke = pypokedex.get(name=p)

try: poke_data = requests.get(f"https://pokemondb.net/pokebase/search?q={poke}")
except: pass
poke_soup = BeautifulSoup(poke_data.content, "html.parser")
questions = poke_soup.findAll("div", attrs = {"class": "qa-q-item-title"})

for question in questions:
    if question.text.strip().startswith("What is a good moveset for"):
        match = re.search(r'"([^"]+)"', str(question.a))
        if match:
            moveset_url = match.group(1)
            
moveset_data = requests.get(f"https://pokemondb.net/pokebase/{moveset_url}")
moveset_soup = BeautifulSoup(moveset_data.content, "html.parser")
movesets = moveset_soup.findAll("div", attrs = {"itemprop":"text"})

for moveset in movesets:
    count = sum(1 for element in ["@","ability","nature"] if element in moveset.text.lower())
    if count >=2:
        moveset = moveset.text
        break

moveset = moveset.split('\n')
for i in moveset:
    if "- " in i:
        extracted_text = i.split("-", 1)[1].strip().replace("/ ","").replace(" ","-")

        if extracted_text.count("-")>=2 and extracted_text not in exceptions:
            hyphen_count = extracted_text.count("-")
            first_hyphen_position = extracted_text.find("-")
            second_hyphen_position = extracted_text.find("-", first_hyphen_position + 1)
            if second_hyphen_position != -1:
                if extracted_text[:second_hyphen_position].count("-")<2:
                    f_moveset.append(extracted_text[:second_hyphen_position])
                if extracted_text[second_hyphen_position + 1:].count("-")<2:
                    f_moveset.append(extracted_text[second_hyphen_position + 1:])
        else:
            f_moveset.append(extracted_text)

for i in f_moveset:
    for game in games:
        try:
            if poke.learns(i.lower(),game):
                ff_moveset.append(i)
            
            if poke.learns(i.lower(),game) and i in exceptions:
                ff_moveset.append(i)            
                
                
            else:
                pass
        except:
            pass
print(set(ff_moveset))


{'Shadow-Ball', 'Sludge-Bomb', 'Giga-Drain', 'Thunderbolt'}
